In [1]:
%%capture
!pip install mlflow
!pip install pyngrok

In [17]:
import mlflow
import subprocess
from pyngrok import ngrok, conf
import getpass

In [18]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
subprocess.Popen(["mlflow", "ui", "--backend-store-uri", MLFLOW_TRACKING_URI])

<Popen: returncode: None args: ['mlflow', 'ui', '--backend-store-uri', 'sqli...>

In [19]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
# mlflow will create an experiment if it doesn't exist
mlflow.set_experiment("duration-prediction-experiment")

<Experiment: artifact_location='/content/mlruns/1', creation_time=1715896345845, experiment_id='1', last_update_time=1715896345845, lifecycle_stage='active', name='duration-prediction-experiment', tags={}>

In [20]:
print("Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth")
conf.get_default().auth_token = getpass.getpass()
port=5000
public_url = ngrok.connect(port).public_url
print(f' * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{port}\"')

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/auth
··········
 * ngrok tunnel "https://7112-35-202-246-121.ngrok-free.app" -> "http://127.0.0.1:5000"


In [21]:
mlflow.set_experiment("BDL Project")

<Experiment: artifact_location='/content/mlruns/2', creation_time=1715896355333, experiment_id='2', last_update_time=1715896355333, lifecycle_stage='active', name='BDL Project', tags={}>

In [7]:
!pip install efficientnet-pytorch

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached 

In [8]:
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader,Dataset
from efficientnet_pytorch import EfficientNet
from torch import nn,optim
import os
import mlflow
from mlflow.models import infer_signature

print(os.getcwd())

/content


In [9]:
import torch
import pandas as pd
from tqdm import tqdm

def load_checkpoint(checkpoint,model,optimizer,lr):
    print("Loading checkpoint...")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

    for param_group in optimizer.param_groups:
        param_group["lr"] = lr

def save_checkpoint(state,filename='checkpt.pth.tar'):
    print("Saving checkpoint...")
    torch.save(state,filename)

# def get_submission(loader, dataset, model):
#     model.eval()
#     predictions = pd.DataFrame()
#     image_id = 1

#     for image, label in tqdm(loader):
#         image = image.to(myconfig.DEVICE)
#         preds = torch.clip(model(image).squeeze(0), 0.0, 96.0)
#         predictions = pd.concat([predictions,pd.Series(preds.detach().numpy()).to_frame().T],ignore_index=True)

#         image_id += 1

#     predictions.to_csv("submission.csv", index=False)
#     model.train()

def get_rmse(loader, model, loss_fn, device):
    model.eval()
    num_examples = 0
    losses = []
    for batch_idx, (data, targets) in enumerate(loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = loss_fn(scores[targets != -1], targets[targets != -1])
        num_examples += scores[targets != -1].shape[0]
        losses.append(loss.item())

    model.train()
    print(f"Loss on val: {(sum(losses)/num_examples)**0.5}")
    return (sum(losses)/num_examples)**0.5

In [10]:
from torch.utils.data import DataLoader,Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class FacialKeypointDataset(Dataset):
    def __init__(self, csv_file, train=True, transform=None):
        super().__init__()
        self.data = pd.read_csv(csv_file)
        self.category_names = ['left_eye_center_x', 'left_eye_center_y', 'right_eye_center_x', 'right_eye_center_y', 'left_eye_inner_corner_x', 'left_eye_inner_corner_y', 'left_eye_outer_corner_x', 'left_eye_outer_corner_y', 'right_eye_inner_corner_x', 'right_eye_inner_corner_y', 'right_eye_outer_corner_x', 'right_eye_outer_corner_y', 'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y', 'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y', 'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y', 'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y', 'nose_tip_x', 'nose_tip_y', 'mouth_left_corner_x', 'mouth_left_corner_y', 'mouth_right_corner_x', 'mouth_right_corner_y', 'mouth_center_top_lip_x', 'mouth_center_top_lip_y', 'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y']
        self.transform = transform
        self.train = train

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, index):
        if self.train:
            image = np.array(self.data.iloc[index, 30].split()).astype(np.float32)
            labels = np.array(self.data.iloc[index, :30].tolist())
            labels[np.isnan(labels)] = -1
        else:
            image = np.array(self.data.iloc[index, 0].split()).astype(np.float32)
            labels = np.zeros(30)

        ignore_indices = labels == -1
        labels = labels.reshape(15, 2)

        if self.transform:
            image = np.repeat(image.reshape(96, 96, 1), 3, 2).astype(np.uint8)
            augmentations = self.transform(image=image, keypoints=labels)
            image = augmentations["image"]
            labels = augmentations["keypoints"]

        labels = np.array(labels).reshape(-1)
        labels[ignore_indices] = -1

        return image, labels.astype(np.float32)

In [11]:
import torch
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
LEARNING_RATE = 8e-6
WEIGHT_DECAY = 1e-3
BATCH_SIZE = 64
NUM_EPOCHS = 100
NUM_WORKERS = 4
CHECKPOINT_FILE = "b0.pth.tar"
PIN_MEMORY = True
SAVE_MODEL = True
LOAD_MODEL = True

train_transforms = A.Compose(
    [
        A.Resize(width=96, height=96),
        A.Rotate(limit=15, border_mode=cv2.BORDER_CONSTANT, p=0.8),
        A.HorizontalFlip(p=0.4),
        A.RandomBrightnessContrast(contrast_limit=0.5, brightness_limit=0.5, p=0.2),
        A.OneOf([
            A.GaussNoise(p=0.8),
            A.CLAHE(p=0.8),
            A.ImageCompression(p=0.8),
            A.RandomGamma(p=0.8),
            A.Posterize(p=0.8),
            A.Blur(p=0.8),
        ], p=1.0),
        A.OneOf([
            A.GaussNoise(p=0.8),
            A.CLAHE(p=0.8),
            A.ImageCompression(p=0.8),
            A.RandomGamma(p=0.8),
            A.Posterize(p=0.8),
            A.Blur(p=0.8),
        ], p=1.0),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.2, border_mode=cv2.BORDER_CONSTANT),
        A.Normalize(mean=[0.4897, 0.4897, 0.4897], std=[0.2330, 0.2330, 0.2330], max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False),
)


val_transforms = A.Compose(
    [
        A.Resize(height=96, width=96),
        A.Normalize(mean=[0.4897, 0.4897, 0.4897], std=[0.2330, 0.2330, 0.2330], max_pixel_value=255.0),
        ToTensorV2(),
    ], keypoint_params=A.KeypointParams(format="xy", remove_invisible=False),
)

In [12]:
def train_one_epoch(loader, model, optimizer, loss_fn, scaler, device):
    losses = []
    loop = tqdm(loader)
    num_examples = 0
    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        scores[targets == -1] = -1
        loss = loss_fn(scores, targets)
        num_examples += torch.numel(scores[targets != -1])
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Loss average over epoch: {(sum(losses)/num_examples)**0.5}")

In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [14]:
train_ds = FacialKeypointDataset(
    csv_file="/content/gdrive/MyDrive/Colab Notebooks/training_new.csv",
    transform=train_transforms
)
train_loader = DataLoader(
    train_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=True
)
val_ds = FacialKeypointDataset(
    transform=val_transforms,
    csv_file="/content/gdrive/MyDrive/Colab Notebooks/val_new.csv"
)
val_loader=DataLoader(
    val_ds,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    shuffle=False
)
# test_ds = FacialKeypointDataset(
#     csv_file="/Users/nikhilanand/JupyterNotebooks/InterIIT2023/CVSelections/test.csv",
#     transform=val_transforms,
#     train=False,
# )

# test_loader = DataLoader(
#     test_ds,
#     batch_size=1,
#     num_workers=NUM_WORKERS,
#     pin_memory=PIN_MEMORY,
#     shuffle=False,
# )

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [22]:
LEARNING_RATE = 8e-5
WEIGHT_DECAY = 1e-4

with mlflow.start_run(run_name="trial2"):

    loss_fn = nn.MSELoss(reduction="sum")
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(1280, 30)
    model = model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler()
    if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
        load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer, LEARNING_RATE)

    params = {
        "model": model,
        "learning_rate": LEARNING_RATE,
        "weight_decay": WEIGHT_DECAY,
        "num_epochs":10
    }
    mlflow.log_params(params)

    for epoch in range(10):
        print("Epoch ",epoch)
        get_rmse(val_loader, model, loss_fn, DEVICE)
        train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)

        if SAVE_MODEL:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            save_checkpoint(checkpoint, filename="b1.pth.tar")

    rmse = get_rmse(val_loader, model, loss_fn, DEVICE)
    # Log the loss metric
    mlflow.log_metric("rmse_loss", rmse)

2024/05/16 21:57:00 WARNING mlflow.utils.validation: Param value 'EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2),...' (14651 characters) is truncated to 6000 characters to meet the length limit.


Loaded pretrained weights for efficientnet-b0
Epoch  0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Loss on val: 51.993571304554024


100%|██████████| 84/84 [00:56<00:00,  1.50it/s]


...

Loss average over epoch: 25.96631550435859
Saving checkpoint...
Epoch  8
Loss on val: 18.973726133116067


100%|██████████| 84/84 [00:57<00:00,  1.46it/s]


Loss average over epoch: 23.157715383136342
Saving checkpoint...
Epoch  9
Loss on val: 18.602223559203903


100%|██████████| 84/84 [00:55<00:00,  1.51it/s]


Loss average over epoch: 20.790774215229984
Saving checkpoint...
Loss on val: 18.150844707226472


In [23]:
LEARNING_RATE = 8e-5
WEIGHT_DECAY = 1e-4

with mlflow.start_run(run_name="trial3"):

    loss_fn = nn.MSELoss(reduction="sum")
    model = EfficientNet.from_pretrained("efficientnet-b0")
    model._fc = nn.Linear(1280, 30)
    model = model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler()
    if LOAD_MODEL and CHECKPOINT_FILE in os.listdir():
        load_checkpoint(torch.load(CHECKPOINT_FILE), model, optimizer, LEARNING_RATE)

    params = {
        "model": model,
        "learning_rate": LEARNING_RATE,
        "weight_decay": WEIGHT_DECAY,
        "num_epochs":10
    }
    mlflow.log_params(params)

    for epoch in range(10):
        if(epoch>=7):
          LEARNING_RATE = 1e-5
          optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)

        print("Epoch ",epoch)
        get_rmse(val_loader, model, loss_fn, DEVICE)
        train_one_epoch(train_loader, model, optimizer, loss_fn, scaler, DEVICE)

        if SAVE_MODEL:
            checkpoint = {
                "state_dict": model.state_dict(),
                "optimizer": optimizer.state_dict(),
            }
            save_checkpoint(checkpoint, filename="b1.pth.tar")

    rmse = get_rmse(val_loader, model, loss_fn, DEVICE)
    # Log the loss metric
    mlflow.log_metric("rmse_loss", rmse)

2024/05/16 22:11:33 WARNING mlflow.utils.validation: Param value 'EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2),...' (14651 characters) is truncated to 6000 characters to meet the length limit.


Loaded pretrained weights for efficientnet-b0
Epoch  0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Loss on val: 52.02158735831046


100%|██████████| 84/84 [00:56<00:00,  1.48it/s]


...

Loss average over epoch: 27.453023814470797
Saving checkpoint...
Epoch  8
Loss on val: 22.029197594284373


100%|██████████| 84/84 [00:59<00:00,  1.41it/s]


Loss average over epoch: 26.954230657256062
Saving checkpoint...
Epoch  9
Loss on val: 23.60144120486168


100%|██████████| 84/84 [00:54<00:00,  1.54it/s]


Loss average over epoch: 26.547189677937865
Saving checkpoint...
Loss on val: 24.015826707653908
